In [1]:
import pandas as pd

file_path = r"C:\Users\LENOVO\OneDrive\AICTE\AI Medical Diagnosis System\Dataset Text Based\prepocessed_hypothyroid.csv"
df = pd.read_csv(file_path)
print(df.info())
print(df.head())

print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   age           3772 non-null   float64
 1   sex           3772 non-null   float64
 2   on thyroxine  3772 non-null   int64  
 3   TSH           3772 non-null   float64
 4   T3 measured   3772 non-null   int64  
 5   T3            3772 non-null   float64
 6   TT4           3772 non-null   float64
 7   binaryClass   3772 non-null   int64  
dtypes: float64(5), int64(3)
memory usage: 235.9 KB
None
    age  sex  on thyroxine   TSH  T3 measured      T3    TT4  binaryClass
0  41.0  1.0             0  1.30            1  2.5000  125.0            0
1  23.0  1.0             0  4.10            1  2.0000  102.0            0
2  46.0  0.0             0  0.98            0  2.0135  109.0            0
3  70.0  1.0             1  0.16            1  1.9000  175.0            0
4  70.0  1.0             0  0.72     

In [2]:
print(df.columns)

Index(['age', 'sex', 'on thyroxine', 'TSH', 'T3 measured', 'T3', 'TT4',
       'binaryClass'],
      dtype='object')


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

# Specify the target column
target_col = "on thyroxine"

# Handle missing values using median imputation
imputer = SimpleImputer(strategy="median")
df[df.columns] = imputer.fit_transform(df)

# Identify categorical columns (excluding the target)
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
if target_col in cat_cols:
    cat_cols.remove(target_col)

# Encode categorical columns
if cat_cols:
    encoder = OrdinalEncoder()
    df[cat_cols] = encoder.fit_transform(df[cat_cols])

# Encode the target column
target_encoder = LabelEncoder()
df[target_col] = target_encoder.fit_transform(df[target_col])

# Separate features and target
X = df.drop(columns=[target_col])
y = df[target_col]

# Standardize only numerical features
num_cols = X.select_dtypes(include=["number"]).columns
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

# Convert back to DataFrame (optional)
X = pd.DataFrame(X, columns=df.drop(columns=[target_col]).columns)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("✅ Data Preprocessing Complete!")


✅ Data Preprocessing Complete!


In [4]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train Support Vector Machine (SVM)
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)

# Train Logistic Regression
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)
log_pred = log_reg.predict(X_test)

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

# Evaluate Models
print("SVM Accuracy:", accuracy_score(y_test, svm_pred))
print("Logistic Regression Accuracy:", accuracy_score(y_test, log_pred))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))

# Display Classification Reports
print("\nSVM Classification Report:\n", classification_report(y_test, svm_pred))
print("\nLogistic Regression Classification Report:\n", classification_report(y_test, log_pred))
print("\nRandom Forest Classification Report:\n", classification_report(y_test, rf_pred))

SVM Accuracy: 0.8768211920529801
Logistic Regression Accuracy: 0.8781456953642384
Random Forest Accuracy: 0.8807947019867549

SVM Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.93       662
           1       0.00      0.00      0.00        93

    accuracy                           0.88       755
   macro avg       0.44      0.50      0.47       755
weighted avg       0.77      0.88      0.82       755


Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.99      0.93       662
           1       0.53      0.09      0.15        93

    accuracy                           0.88       755
   macro avg       0.71      0.54      0.54       755
weighted avg       0.84      0.88      0.84       755


Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.97      0.93       662

C:\anaconda navigator\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\anaconda navigator\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\anaconda navigator\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [5]:
from lightgbm import LGBMClassifier
import joblib

# Train the model (example)
best_lgbm = LGBMClassifier()
best_lgbm.fit(X_train, y_train)  # Ensure X_train and y_train are defined

# Save the trained model
model_filename = "prepocessed_hypothyroid_model.sav"
joblib.dump(best_lgbm, model_filename)
print(f"Model saved as {model_filename}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 371, number of negative: 2646
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000359 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 531
[LightGBM] [Info] Number of data points in the train set: 3017, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122970 -> initscore=-1.964602
[LightGBM] [Info] Start training from score -1.964602
Model saved as prepocessed_hypothyroid_model.sav
